# Busqueda y Filtrado de Datasets

Este notebook muestra como buscar y filtrar datasets en datos.gob.es usando diferentes criterios.

In [ ]:
import sys
sys.path.insert(0, '..')

import asyncio
import json
import pandas as pd

from server import (
    list_datasets,
    search_datasets,
    get_dataset,
    list_themes,
    list_publishers,
)

def get_fn(tool):
    """Obtener la funcion del tool."""
    return tool.fn if hasattr(tool, 'fn') else tool

## 1. Listar Datasets Recientes

In [ ]:
async def listar_recientes():
    fn = get_fn(list_datasets)
    result = await fn(page=0, sort="-modified")
    return json.loads(result)

datasets = await listar_recientes()
print(f"Datasets en pagina: {datasets.get('total_in_page')}")

# Mostrar como DataFrame
df = pd.DataFrame(datasets.get('datasets', []))
df[['title', 'modified', 'distributions_count']].head(10)

## 2. Buscar por Titulo

In [ ]:
async def buscar_por_titulo(texto):
    fn = get_fn(search_datasets)
    result = await fn(title=texto)
    return json.loads(result)

# Buscar datasets sobre empleo
empleo = await buscar_por_titulo("empleo")
print(f"Datasets encontrados: {empleo.get('total_in_page')}")

df_empleo = pd.DataFrame(empleo.get('datasets', []))
df_empleo[['title', 'publisher']].head(10)

## 3. Filtrar por Tematica

In [ ]:
# Primero ver tematicas disponibles
async def ver_tematicas():
    fn = get_fn(list_themes)
    result = await fn()
    return json.loads(result)

themes = await ver_tematicas()
print("Tematicas disponibles:")
for item in themes.get('items', [])[:10]:
    print(f"  - {item.get('_about', '')}")

In [ ]:
async def buscar_por_tema(tema):
    fn = get_fn(search_datasets)
    result = await fn(theme=tema)
    return json.loads(result)

# Buscar datasets de economia
economia = await buscar_por_tema("economia")
print(f"Datasets de economia: {economia.get('total_in_page')}")

## 4. Filtrar por Formato

In [ ]:
async def buscar_por_formato(formato):
    fn = get_fn(search_datasets)
    result = await fn(format=formato)
    return json.loads(result)

# Buscar datasets en formato CSV
csv_data = await buscar_por_formato("csv")
print(f"Datasets con CSV: {csv_data.get('total_in_page')}")

## 5. Busqueda Combinada

In [ ]:
async def busqueda_combinada():
    fn = get_fn(search_datasets)
    # Buscar datasets de salud en formato CSV
    result = await fn(theme="salud", format="csv")
    return json.loads(result)

salud_csv = await busqueda_combinada()
print(f"Datasets de salud en CSV: {salud_csv.get('total_in_page')}")

df_salud = pd.DataFrame(salud_csv.get('datasets', []))
if not df_salud.empty:
    display(df_salud[['title', 'distributions_count']].head())

## 6. Obtener Detalle de un Dataset

In [ ]:
async def detalle_dataset(dataset_id):
    fn = get_fn(get_dataset)
    result = await fn(dataset_id)
    return json.loads(result)

# Obtener detalle del primer dataset encontrado
if salud_csv.get('datasets'):
    uri = salud_csv['datasets'][0].get('uri', '')
    dataset_id = uri.split('/')[-1] if uri else None
    
    if dataset_id:
        detalle = await detalle_dataset(dataset_id)
        print(json.dumps(detalle, indent=2, ensure_ascii=False)[:2000])

## 7. Busqueda Exacta

Usar `exact_match=True` para buscar palabras completas (evita que 'DANA' encuentre 'ciudadana'):

In [ ]:
async def busqueda_exacta(texto):
    fn = get_fn(search_datasets)
    result = await fn(title=texto, exact_match=True)
    return json.loads(result)

# Buscar exactamente 'COVID'
covid = await busqueda_exacta("COVID")
print(f"Datasets con 'COVID' exacto: {covid.get('total_exact_matches', covid.get('total_in_page'))}")

## Siguiente: Analisis de Datos

En el siguiente notebook veremos como descargar y analizar los datos de los datasets.

[03_analisis_datos.ipynb](03_analisis_datos.ipynb)